<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install tensorflow==2.1.0-rc0

In [3]:
pip install keras==2.3.1

In [4]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')

In [0]:

trn=trn.drop(['isFraud'],1)
useful_features = list(trn.iloc[:, 3:55].columns)

In [0]:
categorical_features = [
    'ProductCD',
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
    'addr1', 'addr2',
    'P_emaildomain',
    'R_emaildomain',
    'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'
]
continuous_features=[i for i in list(trn) if i not in categorical_features]


In [0]:

def handle_cont(df,cols):
  for col in cols:
    if df[col].skew()>1:
        df[col]=np.log10(df[col]+1-min(0,min(df[col].fillna(0))))
    mn=df[col].mean()
    std=df[col].std()
    df[col]=(df[col]-mn)/(std+1e-6)
  return df[cols]
continuous_train=handle_cont(trn,continuous_features)
continuous_test=handle_cont(tst,continuous_features)

In [10]:
continuous_train['isna_sum'] = continuous_train.isna().sum(axis=1)
continuous_test['isna_sum'] = continuous_test.isna().sum(axis=1)

continuous_train['isna_sum'] = (continuous_train['isna_sum'] - continuous_train['isna_sum'].mean())/continuous_train['isna_sum'].std()
continuous_test['isna_sum'] = (continuous_test['isna_sum'] - continuous_test['isna_sum'].mean())/continuous_test['isna_sum'].std()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [11]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
continuous_train=reduce_mem_usage(continuous_train)
continuous_test=reduce_mem_usage(continuous_test)

Memory usage of dataframe is 1685.04 MB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Memory usage after optimization is: 421.26 MB
Decreased by 75.0%
Memory usage of dataframe is 1445.79 MB
Memory usage after optimization is: 361.45 MB
Decreased by 75.0%


In [0]:

continuous_train = continuous_train.fillna(0.)
continuous_test = continuous_test.fillna(0.)

In [13]:
def handle_disc(df,cols,num):
  df=df[cols]
  for col in cols:
    df[col]=df[col].astype(str)
    par=min(df[col].nunique(),num)
    vc=list(df[col].value_counts().iloc[:par-1].index)+['Other']
    print(df[col].nunique())
    df[col]=df[col].apply(lambda x:x if x in vc else 'Other')
    print(df[col].nunique())
    print()
    dum=pd.get_dummies(df[col])
    ls=[]
    for i in dum.columns:
      ls.append(str(col)+str(i)+'_dum')
    dum.columns=ls
    df=pd.concat([df,dum],1)
    del([dum])
    gc.collect()
  df=df.drop(cols,1)
  df=df.astype('uint8')
  sp=df.shape[1]
  return df
import gc
cats = handle_disc(pd.concat([tst[categorical_features],trn[categorical_features]],0),categorical_features ,140)

5
5

17091
140

502
140

134
134

5
5

139
139

5
5

442
140

94
94

61
61

61
61

3
3

3
3

3
3

4
4

3
3

3
3

3
3

3
3

3
3



In [14]:
cont=pd.concat([continuous_train,continuous_test],0)
num_shape = continuous_train.shape[1]
cat_shape = cats.shape[1]
X=pd.concat([cont.reset_index(drop=True),cats.reset_index(drop=True)],1)
del([continuous_train,continuous_test,cont,cats])
gc.collect()

0

In [15]:
import keras
import random
import tensorflow as tf
import keras.backend as K

from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.utils.generic_utils import get_custom_objects
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

np.random.seed(42) # NumPy
random.seed(42) # Python

Using TensorFlow backend.


In [0]:
cat=list(X.filter(regex='dum'))
no_dum=[i for i in X.columns if i not in cat]
num_shape=len(no_dum)
cat_shape=len(cat)

In [0]:

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

In [0]:
from keras.layers import concatenate
from keras.optimizers import Adam
def create_model():
    K.clear_session()
    num_inp = Input(shape=(num_shape,))
    cat_inp = Input(shape=(cat_shape,))
    inps = concatenate([num_inp, cat_inp])
    x = Dense(512, activation=custom_gelu)(inps)
    x = Dense(256, activation=custom_gelu)(x)
    x = Dense(512, activation = custom_gelu)(x)
    x = Dropout(.2)(x)
    cat_out = Dense(cat_shape, activation = "linear")(x)
    num_out = Dense(num_shape, activation = "linear")(x)
    model = Model(inputs=[num_inp,cat_inp], outputs=[num_out, cat_out])
    model.compile(
        optimizer=Adam(.05, clipnorm = 1, clipvalue = 1),
        loss=["mse", "mse"]
    )
    return model

In [0]:
model_mse = create_model()
batch_size=2048
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
auto_ckpt = ModelCheckpoint("ae.model", monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min', period=1)
from keras import backend as K


class WarmUpLearningRateScheduler(keras.callbacks.Callback):
    """Warmup learning rate scheduler
    """

    def __init__(self, warmup_batches, init_lr, verbose=0):
        """Constructor for warmup learning rate scheduler

        Arguments:
            warmup_batches {int} -- Number of batch for warmup.
            init_lr {float} -- Learning rate after warmup.

        Keyword Arguments:
            verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpLearningRateScheduler, self).__init__()
        self.warmup_batches = warmup_batches
        self.init_lr = init_lr
        self.verbose = verbose
        self.batch_count = 0
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.batch_count = self.batch_count + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        if self.batch_count <= self.warmup_batches:
            lr = self.batch_count*self.init_lr/self.warmup_batches
            K.set_value(self.model.optimizer.lr, lr)
            if self.verbose > 0:
                print('\nBatch %05d: WarmUpLearningRateScheduler setting learning '
                      'rate to %s.' % (self.batch_count + 1, lr))
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.005)

In [0]:
from keras.utils import Sequence
class DAESequence(Sequence):
    def __init__(self,df,no_dum,frac=0.15,dumm=range(911),batch_size=2048):
        self.batch_size=batch_size
        self.frac=0.15
        self.dumm=dumm
        self.df=df
        self.cat_data=df[dumm].values
        self.num_data=df[no_dum].values
        self.no_dumm=no_dum
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df
        self.idx=[]
        
        
        
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    
    
    
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        
        
        inps=[]
        outs=[]
        output_x=self.data.iloc[idx:last]
        
        
        
        
        data=output_x[self.no_dumm].values
        noise_x=data.copy()
        for i in range(len(self.no_dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=noise_x[frm,i]
            
        inps.append(noise_x)
        outs.append(data)
        
        data=output_x[self.dumm].values
        noise_x=data.copy()
        for i in range(len(self.dumm)):
            to=np.random.randint(0,size,int(size*self.frac))
            frm=np.random.randint(0,size,int(size*self.frac))
            noise_x[to,i]=noise_x[frm,i]
        
        
        
        inps.append(noise_x)
        outs.append(data)
        
        
        
        
        
        return inps,outs

In [21]:


from math import *
gc.collect()
epochs = 10
batch_size=2048
train_gen=DAESequence(X,no_dum,batch_size=batch_size,dumm=cat)
hist = model_mse.fit_generator(train_gen, steps_per_epoch=len(X)//batch_size, epochs=epochs,
                           verbose=1, workers=-1,
                           use_multiprocessing=True,
                              callbacks=[auto_ckpt, warm_up_lr])

Epoch 1/10
535/535 [==============================] - 69s 130ms/step - loss: 0.2066 - dense_5_loss: 0.1917 - dense_4_loss: 0.0149

Epoch 00001: loss improved from inf to 0.20659, saving model to ae.model
Epoch 2/10
535/535 [==============================] - 67s 125ms/step - loss: 0.1597 - dense_5_loss: 0.1483 - dense_4_loss: 0.0114

Epoch 00002: loss improved from 0.20659 to 0.15976, saving model to ae.model
Epoch 3/10
535/535 [==============================] - 66s 124ms/step - loss: 0.1493 - dense_5_loss: 0.1393 - dense_4_loss: 0.0100

Epoch 00003: loss improved from 0.15976 to 0.14936, saving model to ae.model
Epoch 4/10
535/535 [==============================] - 66s 123ms/step - loss: 0.1389 - dense_5_loss: 0.1294 - dense_4_loss: 0.0095

Epoch 00004: loss improved from 0.14936 to 0.13893, saving model to ae.model
Epoch 5/10
535/535 [==============================] - 66s 122ms/step - loss: 0.1265 - dense_5_loss: 0.1151 - dense_4_loss: 0.0114

Epoch 00005: loss improved from 0.13893 t

In [22]:
from tqdm import tqdm
your_new_df=pd.DataFrame()
for i in tqdm([4,5,6]):
    intermediate_layer_model = Model(inputs=model_mse.input, outputs=model_mse.layers[i].output)
    z = pd.DataFrame(intermediate_layer_model.predict([X[no_dum],X[cat]]))
    columns_names = ['Hidden_'+str(i)+'_'+str(l) for l in range(0, z.shape[1])]
    z.columns=columns_names
    your_new_df=pd.concat([your_new_df,z],1)

100%|██████████| 3/3 [03:07<00:00, 62.48s/it]


In [23]:
path = F"/content/gdrive/My Drive/autoenc_features.csv" 
your_new_df=reduce_mem_usage(your_new_df)
your_new_df.to_csv(path)

Memory usage of dataframe is 5357.57 MB
Memory usage after optimization is: 2678.79 MB
Decreased by 50.0%
